In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image

#using numpy
import numpy as np

#for data load or save
import pandas as pd

#visualize some datasets
import matplotlib.pyplot as plt

#check our work directory
import os

In [11]:
LEARNING_RATE = 0.1 # velocidad de aprendizaje
EPOCHS = 10 # Numero de veces que pasamos todo el dataset por la red
BATCH_SIZE = 64 # cantidad de imagenes antes de actualizar el gradiente

#Batch Gradient Descent. Batch Size = Size of Training Set
#Stochastic Gradient Descent. Batch Size = 1
#Mini-Batch Gradient Descent. 1 < Batch Size < Size of Training Set

In [12]:
device = 'cpu'

In [13]:
train_dir = "./data/catsanddogs/train/"
val_dir = "./data/catsanddogs/validation/"
test_dir = "./data/catsanddogs/test/"

In [14]:
train_list = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f[-3:] == 'jpg']
val_list = [os.path.join(val_dir, f) for f in os.listdir(val_dir) if f[-3:] == 'jpg']
test_list = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f[-3:] == 'jpg']

In [15]:
#data Augumentation
data_transforms =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

In [16]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        self.count = len(self.file_list)
        return self.count
    
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).convert('RGB')
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('_')[0]
        if label == 'dog':
            label=1
        elif label == 'cat':
            label=0
            
        return img_transformed,label

In [17]:
train_data = dataset(train_list, transform=data_transforms)
test_data = dataset(test_list, transform=data_transforms)
val_data = dataset(val_list, transform=data_transforms)

In [18]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=BATCH_SIZE, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=BATCH_SIZE, shuffle=True)

In [19]:
from torchvision import models
# cargamos el modelo
model = models.resnet50(pretrained=True)

# Obtenemos los parametros y hacemos que no sean actualizables por el gradiente
for param in model.parameters():
    param.requires_grad = False

# excepto la ultima linea fc (fully connected)
# sera el mismo modelo pero solo con dos salidas, perros y gatos
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.to(device)

/Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/hernanamiune/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|████████████████████████████████████████████████████| 97.8M/97.8M [00:03<00:00, 27.2MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
optimizer = optim.Adam(params = model.parameters(),lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [23]:
for epoch in range(EPOCHS):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))
    
    
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            
            val_output = model(data)
            val_loss = criterion(val_output,label)
            
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)
            
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))

KeyboardInterrupt: 